# Academic Account

In [1]:
import tweepy
import csv
import pandas as pd
import numpy as np

In [4]:
# API keys corresponding to an academic account
api_key = "4OxfMXFZL9yBDYboGSfcjbFsc"
api_secrets = "Ny9LS2gPD7plQy48c3EpQ3UDvVfqrwvJyZLrAJlgEhQtWz3krW"
access_token =  "1581220096529375238-2Kt6eULOqBD7EvtpqTf4gvU7n69MZ3"
access_secret = "7irLEy7IiGfW72gsh895wntdk3n94upoyXtUMGSo98mch"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPEbiQEAAAAAuuFvB9xcB%2FR4J2RXWNApbAWQ3PY%3DVqvJvvr5WlYDYSfMq5GcZqGGu2gapcfF0ezvJzCWfDaodqSE4L"
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)
try:
    api.verify_credentials()
    print('Successful Authentication')
    #commented so it doesn't appear in hdfs
except:
    print('Failed authentication')

Successful Authentication


In [5]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAPEbiQEAAAAAuuFvB9xcB%2FR4J2RXWNApbAWQ3PY%3DVqvJvvr5WlYDYSfMq5GcZqGGu2gapcfF0ezvJzCWfDaodqSE4L')


Searching Twitter

Search for whatever trait your community has in common. In this case, I searched for a popular Twitter hashtag, limiting the results to the 4000 most recent Tweets.


In [6]:
hashtag = api.search_tweets(q='#yuan')

In [7]:
#show contents of search
for tweet in hashtag:
    print(tweet.text)
      


RT @KitcoNewsNOW: Ex-Goldman chief economist calls on BRICS to challenge USD's dominance as China leads de-dollarization trend https://t.co…
FIRSTPOST

Saudi Arabia Enters China-Led SCO: US' Hegemony in West Asia Over?

https://t.co/7INThitz9d

#USDollar… https://t.co/JgApVFIDp7
Eğer Çin/Rusya'nın #yuan adımına seviyorsanız bir söz ileteyim size;

Amaç #ABD'yi bitirmek ise, Tüm dünya ABD'ye a… https://t.co/cuhyuUlWdH
RT @andrewliberty23: #China settles first LNG trade in #yuan.

#Brazil-China trade agreement substitutes dollars for yuan and reals.

My fu…
RT @KitcoNewsNOW: Ex-Goldman chief economist calls on BRICS to challenge USD's dominance as China leads de-dollarization trend https://t.co…
Ex-Goldman chief economist calls on BRICS to challenge USD's dominance as China leads de-dollarization trend… https://t.co/w2niaaNBeu
RT @BPartisans: BOUM !!!!!
La France signe un accord non libellé en USD sur le commerce du GNL.
RIP dollar américain.

#China et #France To…
RT @andrewliberty23: #

In [8]:
# Create lists for each field desired from the tweets.
sn = []
text = []
timestamp =[]
for tweet in hashtag:
    #print (tweet.user.screen_name, tweet.created_at, tweet.text)
    timestamp.append(tweet.created_at)
    sn.append(tweet.user.screen_name)
    text.append(tweet.text)
    


In [9]:
# Convert lists to dataframe
df = pd.DataFrame()
df['timestamp'] = timestamp
df['sn'] = sn
df['text'] = text


In [10]:
# Prepare for date filtering. Adding an EST time column since chat hosted by people in that time zone.
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['EST'] = df['timestamp'] - pd.Timedelta(hours=5) #Convert to EST

df['EST'] = pd.to_datetime(df['EST'])


In [11]:
df.head()

,timestamp,sn,text,EST
0,2023-03-30 17:38:45+00:00,EllieLove2019,RT @KitcoNewsNOW: Ex-Goldman chief economist c...,2023-03-30 12:38:45+00:00
1,2023-03-30 17:37:20+00:00,THEENDGAMEWAR1,FIRSTPOST\n\nSaudi Arabia Enters China-Led SCO...,2023-03-30 12:37:20+00:00
2,2023-03-30 17:36:58+00:00,FatihthePau,Eğer Çin/Rusya'nın #yuan adımına seviyorsanız ...,2023-03-30 12:36:58+00:00
3,2023-03-30 17:36:58+00:00,dizzy_lizzyy1,RT @andrewliberty23: #China settles first LNG ...,2023-03-30 12:36:58+00:00
4,2023-03-30 17:35:48+00:00,CDeutschmanek,RT @KitcoNewsNOW: Ex-Goldman chief economist c...,2023-03-30 12:35:48+00:00


In [12]:
# Write out Tweets in case they are needed later.
df.to_csv('yuantweets.csv',index = False,encoding='utf-8')


In [13]:

# Create a list of the unique usernames in order to see which users we need to retrieve friends for.
allNames = list(df['sn'].unique())




Retrieve User Information

Now having a list of all of the users we would like to include in our community, we can retrive additional necessary information such as who their are following in order to build our social graph.

Note that Twitter does have strict rate limits that can cause problems at this point. A timeout is built into every iteration to minimize this. If problems still occur, you may want to include an intermediate write out on every iteration to maintain what has been captured to that point. The loop can be restarted from the nth value of allNames where a break occurs (i.e. "for name in allNames[n:]").


In [14]:

# Initialize dataframe of users that will hold the edge relationships
dfUsers = pd.DataFrame()
dfUsers['userFromName'] =[]
dfUsers['userFromId'] =[]
dfUsers['userToId'] = []
count = 0 

nameCount = len(allNames)


In [18]:
import sys 
import time

In [19]:

# The choice to retrieve friends (who the user is following) rather than followers is intentional.
# Either would work. However, many Twitter users follow fewer users than are following them, especially the most popular accounts. 
# This reduces the number of very large calls to Twitter API, which seemed to cause problems.
for name in allNames:
    # Build list of friends    
    currentFriends = []
    for page in tweepy.Cursor(api.get_friends, screen_name=name).pages(2):
        currentFriends.extend(page)
    currentId = api.get_user(screen_name=name).id
    currentId = [currentId] * len(currentFriends)
    currentName = [name] * len(currentFriends)   
    dfTemp = pd.DataFrame()
    dfTemp['userFromName'] = currentName
    dfTemp['userFromId'] = currentId
    dfTemp['userToId'] = currentFriends
    dfUsers = pd.concat([dfUsers,dfTemp])
    time.sleep(70) # avoids hitting Twitter rate limit
    # Progress bar to track approximate progress
    count +=1
    per = round(count*100.0/nameCount,1)
    sys.stdout.write("\rTwitter call %s%% complete." % per)
    sys.stdout.flush()    
    


Twitter call 13.3% complete.

In [ ]:
# Again, to limit the number of calls to Twitter API, just do lookups on followers that connect to those in our user group.
# We are not interested in "friends" that are not part of this community.
fromId = dfUsers['userFromId'].unique()
dfChat = dfUsers[dfUsers['userToId'].apply(lambda x: x in fromId)]

# No more Twitter API lookups are necessary. Create a lookup table that we will use to get the verify the userToName
dfLookup = dfChat[['userFromName','userFromId']]
dfLookup = dfLookup.drop_duplicates()
dfLookup.columns = ['userToName','userToId']
dfCommunity = dfUsers.merge(dfLookup, on='userToId')

dfCommunity.to_csv('dfCommunity.csv',index = False,encoding='utf-8')


KeyError: "None of [Index(['userFromName', 'userFromId'], dtype='object')] are in the [columns]"